In [4]:
import requests
from pathlib import Path

from bs4 import BeautifulSoup

In [329]:
base_url = 'https://www.erowid.org'
# S=2 specifies LSD; C=1 specifies general (Vs. first times, combinations, glowing experiences etc)
lsd_index_url = 'https://www.erowid.org/experiences/exp.cgi?S=2&C=1&Start=0&Max=1000'
# lsd_index_url = 'experiences/subs/exp_LSD_General.shtml'
index_html_path = "base_html.html"

In [266]:
if Path(index_html_path).exists():
    with open(index_html_path, 'r') as fhand:
        html_doc = fhand.read()
else:
    request = requests.get(lsd_index_url, auth=('user', 'pass'))
    request.status_code
    html_doc = request.text
    with open(index_html_path, 'w') as fhand:
        fhand.write(html_doc)

In [268]:
# soup = BeautifulSoup(html_doc, 'html.parser')
soup = BeautifulSoup(html_doc, 'lxml')
print(soup.prettify()[:500])

<html>
 <head>
  <title>
   LSD Reports - General : Erowid Experience Vaults
  </title>
  <meta content="Erowid Experience Vaults: An Experience" name="description"/>
  <meta content="Experience Report Vaults, trip reports, stories, descriptions" name="keywords"/>
  <link href="/includes/general_default.css" rel="stylesheet" type="text/css"/>
  <link href="includes/exp.css" rel="stylesheet" type="text/css"/>
  <!-- Sperowider <noindex/> -->
 </head>
 <body alink="#008080" bgcolor="#000000" link=


In [269]:
tables = soup.find_all('table')

# We want the third table (which is actually nested in the second)
table = tables[2]

In [285]:
list(table.children)[:6]

['\n', <tr height="10">
 <th width="75"><input onclick="SortBy('RA');" type="Button" value="Rating"/><img src="/experiences/images/arrow_down.jpg"/></th>
 <th width="230"><input onclick="SortBy('TA');" type="Button" value=" Title"/></th>
 <th width="105"><input onclick="SortBy('AA');" type="Button" value=" Author "/></th>
 <th width="150"><input onclick="SortBy('SA');" type="Button" value="Substance"/></th>
 <th width="85"><input onclick="SortBy('PDD');" type="Button" value="Pub Date"/><img src="/experiences/images/arrow_down.jpg"/></th>
 </tr>, '\n', <tr height="8"><th colspan="5"></th></tr>, '\n', <tr class=""><td> <img align="right" alt="Very Highly Recommended" border="0" src="images/exp_star_3.gif"/></td><td><a href="exp.php?ID=89042">Some Growing Up to Do</a></td><td>thingummajig</td><td>MDMA, LSD &amp; Cannabis</td><td align="right">Feb 2 2012</td>
 </tr>]

In [292]:
# Get all rows of this table
rows = table.find_all('tr')

In [287]:
# First row is the headers - we will retrieve the header values
headers = rows[0].find_all('th')
headers

[<th width="75"><input onclick="SortBy('RA');" type="Button" value="Rating"/><img src="/experiences/images/arrow_down.jpg"/></th>,
 <th width="230"><input onclick="SortBy('TA');" type="Button" value=" Title"/></th>,
 <th width="105"><input onclick="SortBy('AA');" type="Button" value=" Author "/></th>,
 <th width="150"><input onclick="SortBy('SA');" type="Button" value="Substance"/></th>,
 <th width="85"><input onclick="SortBy('PDD');" type="Button" value="Pub Date"/><img src="/experiences/images/arrow_down.jpg"/></th>]

In [288]:
col_names = [th.input['value'].strip() for th in headers]
col_names

['Rating', 'Title', 'Author', 'Substance', 'Pub Date']

In [301]:
rows[0]

<tr class=""><td> <img align="right" alt="Very Highly Recommended" border="0" src="images/exp_star_3.gif"/></td><td><a href="exp.php?ID=89042">Some Growing Up to Do</a></td><td>thingummajig</td><td>MDMA, LSD &amp; Cannabis</td><td align="right">Feb 2 2012</td>
</tr>

In [300]:
# Iterate through the remaining rows - 2nd row is empty
rows = rows[2:]

In [302]:
all_links = []

for row in rows:
    entry = {}
    # Get all cells of the row
    for i, td in enumerate(row.find_all('td')):
        entry[col_names[i]] = td.text
        # If link included, add that to dict
        if td.find('a') is not None:
            entry['href'] = td.find('a').get('href')
    all_links.append(entry)

In [303]:
len(all_links)

648

In [312]:
all_links[:3]

[{'Rating': ' ',
  'Title': 'Some Growing Up to Do',
  'href': 'exp.php?ID=89042',
  'Author': 'thingummajig',
  'Substance': 'MDMA, LSD & Cannabis',
  'Pub Date': 'Feb 2 2012'},
 {'Rating': ' ',
  'Title': 'My Minidose Manifesto',
  'href': 'exp.php?ID=112505',
  'Author': 'Uncle Iroh',
  'Substance': 'LSD',
  'Pub Date': 'Oct 26 2018'},
 {'Rating': ' ',
  'Title': 'The Colossus',
  'href': 'exp.php?ID=112152',
  'Author': 'nervewing',
  'Substance': 'Memantine, 3-MEO-PCE, LSD, 4-AcO-MiPT, 4-HO-MET, 2C-C, Clonazepam & Aripiprazole',
  'Pub Date': 'Aug 4 2018'}]

Many reviews seem to include multiple drugs, e.g. the first review includes cannabis, and the third review would appear to include memantine, 3-MEO-PCE and a bunch of things. We'll restrict to just the pure LSD reviews.

In [307]:
lsd_only = [ent for ent in all_links if ent['Substance'] == 'LSD']

In [308]:
len(lsd_only)

433

In [313]:
lsd_only[:3]

[{'Rating': ' ',
  'Title': 'My Minidose Manifesto',
  'href': 'exp.php?ID=112505',
  'Author': 'Uncle Iroh',
  'Substance': 'LSD',
  'Pub Date': 'Oct 26 2018'},
 {'Rating': ' ',
  'Title': 'Physics at the Edge of the Universe',
  'href': 'exp.php?ID=69866',
  'Author': 'Spooky',
  'Substance': 'LSD',
  'Pub Date': 'Apr 19 2016'},
 {'Rating': ' ',
  'Title': 'Somatic Vision and Cosmic Consciousness',
  'href': 'exp.php?ID=77462',
  'Author': 'Lokapalas',
  'Substance': 'LSD',
  'Pub Date': 'Nov 28 2013'}]

In [335]:
for i in range(10):
    print(f"{i + 1}. {lsd_only[i]['Title']}")

1. My Minidose Manifesto
2. Physics at the Edge of the Universe
3. Somatic Vision and Cosmic Consciousness
4. LSD Microdosing RCT
5. It Can Be Whatever I Want It to Be
6. Arrested
7. So Sure I Could Handle It
8. Sipping on the World Syrup
9. A Life Cycle
10. Breakthrough on Bicycle Day


In [330]:
url_to_grab = base_url + '/experiences/' + 'exp.php?ID=112505'
url_to_grab

In [336]:
request = requests.get(url_to_grab, auth=('user', 'pass'))

In [339]:
html = BeautifulSoup(request.text, 'lxml')
print(html.prettify()[:500])

<html>
 <head>
  <title>
   LSD - Erowid Exp - 'My Minidose Manifesto'
  </title>
  <meta content="An Experience with LSD. 'My Minidose Manifesto' by Uncle Iroh" name="description"/>
  <meta content="Experience Report Vaults drug trip reports stories descriptions" name="keywords"/>
  <link href="/includes/general_default.css" rel="stylesheet" type="text/css"/>
  <link href="includes/exp_view.css" id="main_css" rel="stylesheet" type="text/css"/>
  <link href="includes/exp_view_light_on_dark.css" 


In [343]:
divs = html.find_all('div')
len(divs)

15

In [354]:
dicty = {}
for div in divs:
    if 'class' in div.attrs:
        # Returns class as list, to allow multi-classes
        if 'title' in div['class']:
            dicty['title'] = div.text
        elif 'substance' in div['class']:
            dicty['substance'] = div.text
        elif 'author' in div['class']:
            dicty['author'] = div.a.text
        # Main body or report stored here
        elif 'report-text-surround' in div['class']:
            dicty['main'] = div
        
dicty.keys()

dict_keys(['title', 'substance', 'author', 'main'])

In [360]:
lsd_only[0]

{'Rating': ' ',
 'Title': 'My Minidose Manifesto',
 'href': 'exp.php?ID=112505',
 'Author': 'Uncle Iroh',
 'Substance': 'LSD',
 'Pub Date': 'Oct 26 2018'}

In [364]:
display(dicty['title'])
display(dicty['author'])
display(dicty['substance'])

'My Minidose Manifesto'

'Uncle Iroh'

'LSD'

Good, looks like we have the correct review.

In [355]:
main = dicty['main']

In [357]:
tables = main.find_all('table')

In [359]:
# First table doesn't seem to have anything
# Second table has dose and other info
tables[1]

<table border="2" bordercolor="#224422" cellpadding="4" cellspacing="0" class="dosechart">
<tr>
<td align="right" width="90">DOSE:<br/></td>
<td align="center" class="dosechart-amount" width="90">10-15 ug</td>
<td align="center" class="dosechart-method">oral</td>
<td class="dosechart-substance"><a href="/chemicals/lsd/">LSD</a></td>
<td class="dosechart-form"><b>(blotter / tab)</b></td>
</tr>
</table>